# Thomas Wildfire Perimeter and Air Quality Index
Hwk 2-Task 2: Air Quality Index Data + Hwk4-Task 2: False color image

Author: Vanessa Salgado

Link to GitHub repository: https://github.com/Vanessa-Salgado/eds220-hw4-task3

## Purpose
The Thomas Fires was a massife wildfire that affected Ventura and Santa Barbara counties in December 2017. 
The Thomas Fire destroyed at least 1,063 structures, while damaging 280 others; and the fire caused over $2.2 billion (2018 USD) in damages. As of August 2020, the Thomas Fire is California's tenth-most destructive wildfire. In this notebook we analyze the Air Quality Index that occured the time leading up to and after the Thomas Fire. We will create a map of the Thomas Fire perimeter to visualize the damage using `landsat 8` imagery 

### Highlights

* Data wrangling and exploration of with pandas
* Geospatial data wrangling with geopandas
* Merging of tabular and vector data
* Creating and customizing raster map 

### Dataset Description

In this notebook we use N datasets: 

**1) US EPA Air Quality Index(AQI)**

The US Environmentla Protection Agency air quality collects data from outdoor monitors across the United States, Puerto Rico, and the U.S Virgin Islands. The data comes primarily from the AQS (Air Quality System) database.

This data set was downloaded from the EPA AirData portal. The data was available from the Pre-Generate Data Files that collects daily AQI readings by county per year [https://aqs.epa.gov/aqsweb/airdata/download_files.html#AQI]. The files are updated twice per year: once in June to capture the complete data for the prior year and once in December to capture the data for the summer (ozone season). For this analysis we collect the will use the following columns of interest: 

* date: 
* county Name: 
* Date: 
* AQI: 
* 

We specifically look at 2017 and 2018 in order to study the before and after affects of the Thomas Fire

Access here: 

**2)Landsat 8 Surface Reflectance Satellite Data**

A simplified collection of bands (red, green, blue, near-infrared and shortwave infrared) from the Landsat Collection 2 Level-2 atmosperically corrected surface reflectance data, collected by the Landsat 8 satellite.

The data was accessed and pre-processed in the Microsoft Planetary Computer to remove data outside land and coarsen the spatial resolution (Landsat Collection in MPC). Data should be used for visualization purposes only.

Access here: 

**3) 

A shapefile of fire perimeters in California during 2017. The state of California contains geospatial data of  California's Fire Wildfire Perimeters and Prescribed Burns hosted on the CAL FIRE AGeoportal. You can access it here: 

## Final Output


In [ ]:
# TBD

## Data Exploration

#### Import Libraries and functions:  

In [ ]:
#import libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import xarray as xr
import rioxarray as rioxr
import geopandas as gpd
from rasterio.features import rasterize

#### Import Data

In [2]:
#import 2017 and 2018 aqi data 
aqi_18 = pd.read_csv('https://aqs.epa.gov/aqsweb/airdata/daily_aqi_by_county_2018.zip')

NameError: name 'pd' is not defined

#### Prepare Data



In [ ]:
# combine 2017 and 2018 into a single df using concatenation
aqi = pd.concat([aqi_17, aqi_18])

# 

Clean and make datatypes of columns appropaite to perform arithmetic functions: 

In [ ]:
# combine 2017 and 2018 into a single df using concatenation
aqi = pd.concat([aqi_17, aqi_18])

#clean the column names, removing spaces and turning all strings to lowercase
aqi.columns = aqi.columns.str.lower().str.replace(' ','_') #code provided

# create subset of SB County w/desired columns
aqi_sb = aqi.loc[aqi['county_name']=='Santa Barbara', #row-selection
       ['date','aqi','category','defining_parameter', 'defining_site','number_of_sites_reporting']] #column-selection

# change date column to object type = datatime
aqi_sb.date = pd.to_datetime(aqi_sb.date)

## Analysis
Analysis
- Include subsections as necessary to guide reader through your analysis
- Include checks to see operations worked
- Checks must be short and informative: print specific attributes

Inspecting the the daily AQI of Santa Barbara County would introduce a lot of noise into the analysis. A rolling 5 day average would seem like an appropiate modification to analyzing daily AQI readings. 

### Fetch data 

### Raster Visualization
